In [ ]:
from boardlaw.arena.multi import *
evaluate()


In [ ]:
!rm -rf /root/.cache/torch_extensions/

In [ ]:
boardsize = 7

agents = sql.agent_query().query(f'description == "bee/{boardsize}" & test_nodes == 64 & width >= 128')
trials = (sql.trial_query(boardsize, 'bee/%')
            .loc[lambda df: df.black_agent.isin(agents.index)]
            .loc[lambda df: df.white_agent.isin(agents.index)])

In [ ]:
_, games = elos.symmetrize(trials)
games = games.reindex(index=agents.index, columns=agents.index).fillna(0)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(games)

In [ ]:
sql.agent_query().query(f'description == "bee/{boardsize}" & test_nodes == 64 & width <= 128')

In [ ]:
import gc
ptrs = []
for obj in gc.get_objects(2):
    try:
        if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
            ptrs.append(obj.data_ptr())
    except:
        pass
new_ptrs = set(ptrs)

In [ ]:
new_ptrs - old_ptrs

In [ ]:
boardsize = 9
agents = sql.agent_query().query(f'description == "bee/{boardsize}" & test_nodes == 64 & width >= 512')
trials = (sql.trial_query(9, 'bee/%')
            .loc[lambda df: df.black_agent.isin(agents.index)]
            .loc[lambda df: df.white_agent.isin(agents.index)])

In [ ]:
sizes = 4 * agents.width**2 * agents.depth
n_games_per = 512
chunks = []
chunk = []
for current in sizes.index:
    size = len(chunk)*n_games_per*sizes[chunk].sum()
    if size > 1e9:
        chunks.append(chunk)
        chunk = []
    chunk.append(current)

In [ ]:
sizes/1e6